In [ ]:
# ── 설치(최초 1회) ─────────────────────────────────────────────
!pip -q install -U langchain langchain-openai langsmith


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 6.9 MB/s eta 0:00:00


In [ ]:
# 1) (선택) OpenAI 사용 시
import os, uuid
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "llm_colab_ex_4"                 # 수업용 프로젝트명


In [ ]:

# ── 임포트 ───────────────────────────────────────────────────
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnablePassthrough

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# ── 1) 짧은 경로(원샷 요약) ───────────────────────────────────
prompt_short = ChatPromptTemplate.from_template(
    "다음 텍스트를 한 문단으로 간결히 요약하라:\n\n{text}"
)
short_chain = (
    {"text": RunnableLambda(lambda x: x["text"])}
    | prompt_short
    | llm
    | parser
)

# ── 2) 긴 경로(Map-Reduce 요약) ──────────────────────────────
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=80)

# map 단계: 청크별 요약
prompt_map = ChatPromptTemplate.from_template(
    "다음 청크를 한 문장으로 요약하라:\n\n{chunk}"
)
map_chain = prompt_map | llm | parser

# reduce 단계: 부분요약들을 통합
prompt_reduce = ChatPromptTemplate.from_template(
    "다음 부분 요약들을 통합해 한 문단으로 명확히 요약하라:\n\n{summaries}"
)
reduce_chain = prompt_reduce | llm | parser

# 긴 경로 체인
long_chain = (
    # 1) 텍스트 가져오기
    {"text": RunnableLambda(lambda x: x["text"])}
    # 2) 청킹
    | RunnableLambda(lambda d: splitter.split_text(d["text"]))
    # 3) 청크 리스트 → [{chunk: ...}, ...]로 변환
    | RunnableLambda(lambda chunks: [{"chunk": c} for c in chunks])
    # 4) 각 청크에 map_chain 적용 → 부분 요약 리스트
    | map_chain.map()
    # 5) 부분 요약을 합쳐 reduce 입력 만들기
    | RunnableLambda(lambda partials: {"summaries": "\n".join(partials)})
    # 6) reduce 실행
    | reduce_chain
)

# ── 3) 라우터: 길이 기준 분기 (예: 400자 이하 = 짧은 경로) ───
router = RunnableBranch(
    (lambda x: len(x.get("text","")) <= 400, short_chain),
    long_chain  # else
)

# ── 4) 실행 ───────────────────────────────────────────────────
short_text = "LangSmith는 LLM 앱을 관찰/평가/디버깅하는 플랫폼이다. 개발팀은 체인 단계별 로그를 추적해 품질을 개선한다."
long_text = """\
LangSmith는 LLM 애플리케이션을 관찰하고 평가하며 디버깅할 수 있게 도와주는 도구다.
대화형 에이전트나 RAG 파이프라인을 만들 때, LangSmith를 통해 체인 단계를 투명하게 기록하고,
어디서 실패하는지, 어떤 프롬프트/도구 호출이 비효율적인지 추적할 수 있다.
길이가 긴 문서는 보통 청킹을 해서 각 청크를 요약한 뒤, 부분 요약들을 통합하는 방식(map-reduce)으로 요약 품질과 추론 속도를 모두 확보한다.
"""

cfg = {"tags":["demo","routing","summarization"], "run_name":"length_based_summarizer"}

print("▶ 짧은 텍스트 요약:")
print(router.invoke({"text": short_text}, config=cfg))

print("\n▶ 긴 텍스트 요약:")
print(router.invoke({"text": long_text}, config=cfg))


▶ 짧은 텍스트 요약:
LangSmith는 LLM 앱을 관찰, 평가 및 디버깅하는 플랫폼으로, 개발팀이 체인 단계별 로그를 추적하여 품질을 개선하는 데 도움을 준다.

▶ 긴 텍스트 요약:
LangSmith는 LLM 애플리케이션의 관찰, 평가, 디버깅을 지원하는 도구로, 대화형 에이전트나 RAG 파이프라인을 구축할 때 체인 단계를 기록하고 실패 지점 및 비효율적인 프롬프트/도구 호출을 추적할 수 있게 해준다. 긴 문서는 청킹하여 각 청크를 요약한 후, 이를 통합하는 방식으로 요약 품질과 추론 속도를 향상시킨다.
